In [1]:
import scipy
import numpy as np
import networkx as nx
import ot
import time
import multiprocessing as mp
from tqdm import tqdm, trange
import torch
import cvxpy as cp
import torch.nn as nn
from torch.optim import Adam, SGD
import matplotlib.pyplot as plt

In [12]:
class PointNetLearner(nn.Module):
    def __init__(self, h_in, h_out, g_out, num_h_layers=10, num_g_layers = 10):
        super(PointNetLearner, self).__init__()
        self.h = []
        self.h.append(nn.Linear(h_in, h_out))
        for i in range(num_h_layers - 1):
            self.h.append(nn.Linear(h_out, h_out))
        
        self.gamma = []
        self.gamma.append(nn.Linear(2*h_out, g_out))
        for i in range(num_g_layers - 2):
            self.gamma.append(nn.Linear(g_out, g_out))
        self.gamma.append(nn.Linear(g_out, 1))

    def forward(self, P, Q):
        outh_P = P
        for layer in self.h:
            outh_P = torch.sigmoid(layer(outh_P))
        outh_P = torch.sum(outh_P, axis=0)
        
        outh_Q = Q
        for layer in self.h:
            outh_Q = torch.sigmoid(layer(outh_Q))
        outh_Q = torch.sum(outh_Q, axis=0)

        gamma_in_vec = torch.cat((outh_P, outh_Q))
        print(gamma_in_vec.size())

        res = gamma_in_vec
        for layer in self.gamma:
            res = torch.sigmoid(layer(res))
        return res


In [16]:
P = 5*np.random.uniform(size=(6, 2))
Q = 5*np.random.uniform(size=(6, 2))
mat = ot.dist(P, Q)
a = (1/6) * np.ones(6)
dist = ot.emd2(a, a, mat)

P =torch.tensor(P, dtype=torch.float32)
Q = torch.tensor(Q, dtype=torch.float32)


model = PointNetLearner(2, 10, 10)
print(model.parameters())
optimizer = torch.optim.Adam([model.parameters()], lr=0.001)
for i in range(5):
    optimizer.zero_grad()
    y = model(P, Q)
    loss = np.square(y - dist)
    loss.backward()
    optimizer.step()
    print(y, dist)

<generator object Module.parameters at 0x7fc1f4e88dd0>


TypeError: optimizer can only optimize Tensors, but one of the params is Module.parameters